## Les bryggeridata

In [14]:
import geopandas as gpd

In [15]:
#funksjoner for å fikse opp dataene

#oversett alt annet enn punkt til punkt
def to_point(row):
    if row['geometry'].geom_type == 'Point':
        return row['geometry']
    return row['geometry'].centroid

#set operator-taggen
def fix_operator(row):
    if row['operator'] is None:
        return row['name']
    return row['operator']

In [16]:
# les inn bryggeriene fra en geojson-fil
bryggerier = gpd.read_file('./data/osm_breweries_norway_08032018.geojson')

#fiks opp kolonner
bryggerier['geometry'] = bryggerier.apply(to_point, axis=1)
bryggerier['operator'] = bryggerier.apply(fix_operator, axis=1)

# velg ut de kolonnene vi faktisk e
bryggerier = bryggerier[['name', 'operator', 'geometry']]


In [19]:
bryggerier

,name,operator,geometry
0,Lervig Aktiebryggeri,Lervig Aktiebryggeri,POINT (5.746142767034293 58.94570846949821)
1,Ringnes Bryggeri,Ringnes Bryggeri,POINT (10.93189032774258 59.99209675567997)
2,Borg Bryggerier,Hansa Borg,POINT (11.11239512048502 59.29057490401918)
3,Haandbryggeriet,Haandbryggeriet,POINT (10.23138505977879 59.74216480250969)
4,E. C. Dahls Bryggeri,Ringnes Bryggeri,POINT (10.42700137022198 63.44091298857254)
5,Christianssand Brygghus,Christianssand Brygghus,POINT (7.993930033407581 58.14454205074)
6,Christianssands Bryggeri,Hansa Borg,POINT (7.979171369885761 58.14782034059565)
7,Aass bryggeri,Aass bryggeri,POINT (10.21120833326642 59.7414731548904)
8,Little Brother Brewery,Little Brother Brewery,POINT (10.74483330963815 59.91937024420984)
9,Sagene Bryggeri,Sagene Bryggeri,POINT (10.7563746337984 59.93101542900136)
